# 서울스토어 크롤러3

분류용 상품 이미지를 크롤링하기 위한 메타 정보 크롤러입니다. a.k.a "통" 만들기 사전작업

서울스토어의 카테고리별 상품 정보(이미지, 상세정보 링크)를 크롤링하는 크롤러입니다. 크롤링 결과를 활용해서 상품 메타정보를 저장 + 상품 이미지(별도의 크롤러 활용)를 저장합니다.

크롤링한 카테고리는 아래와 같습니다. (category number)
- 상의 (1251)
- 하의 (1253)
- 원피스 (1252)

크롤링 결과는 다음의 형태로 json 파일로 저장합니다.
[{상품1}, {상품2}, ...]
{상품1} = {"key": 상품번호, "product_url": 상품url, "img_url": 이미지url}

## Histroy

200517 다영
- url별 다른 파일로 저장 (진행전)
- 선 스크롤 후 리스트 수집
- PAGE_DOWN 대신 END 사용
- 가끔 오류로 스크롤이 안 될 경우 예외 처리
- 목표 갯수 미만 & 스크롤이 더 이상 안 될 때 예외 처리
- 저장 파일명에 현재 시간 추가
- json 저장 대신 csv로 선 저장 후 json으로 변환 (진행전)
- 리스트 상 보이는 정보는 모두 저장
- 카테고리 컬럼에는 사이트가 사용하는 카테고리 넘버 저장

In [15]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [16]:
#카테고리별 url(상의, 하의, 원피스)
root_urls = ['https://www.seoulstore.com/categories/1251/regDatetime/desc',
            'https://www.seoulstore.com/categories/1253/regDatetime/desc',
            'https://www.seoulstore.com/categories/1252/regDatetime/desc']

In [ ]:
for url in root_urls:  # 인자로 주어진 url 하나씩 취함
    data_temp = get_products(url, num, wait_time)
    output(get_products(data_temp, 100, 60), 'tong_100',True)

In [19]:
def get_products(url, num, wait_time):
    """
    Arguments:
    - urls: 리스트. 크롤링할 url들을 원소로 함.
    - num: url당 크롤링할 아이템 수
    - wait_time: 로딩을 몇 초까지 기다릴지

    Return:
    - total_posts: 리스트. 아이템 당 {"product_url": 상품url, "img_url": 이미지url} 정보를 크롤링한 것을 원소로 함.

    크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
    """
    
    total_posts = []         # 전체 크롤링 결과를 저장할 리스트
    product_set = set()   # 중복 크롤링 거르기 위한 셋. product_url을 원소로 함
    unit_of_scroll = 10      # 반복할 스크롤 단위
    count_nolonger = 0  # 목표 갯수 미만 & 스크롤이 더 이상 안 될 때를 대비한 카운트 변수
    
    
    # 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨김
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(
        '--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
    # 크롬 브라우저 실행
    browser = webdriver.Chrome(executable_path='chromedriver_81', chrome_options=chrome_options)
    # 대기가능한 최대 시간
    wait = WebDriverWait(browser, wait_time)

    posts = []  # 당해 url 크롤링 결과를 저장할 리스트
    
    browser.get(url)
    body = browser.find_element_by_tag_name('body')
    
    #카테고리 값
    category = url.split('categories/')[-1].split('/')[0]
#    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'image_container')))
#    category = browser.find_element_by_class_name('Category_Nav').find_elements_by_class_name('active')[0].text
    
    
    # 초기에 목표값만큼 스크롤 - 갈수록 속도가 느려져서 슬립타임 세분화
    count_temp = 0
    for i in range(num // 12):
        body.send_keys(Keys.END)
        if i < 40 : time.sleep(0.5)
        elif 40<=i<200 : time.sleep(1)
        else : time.sleep(2)
        count_temp += 1
    
    # 목표 갯수 도달까지 스크롤
    breakpoint = 0
    while len(posts) < num and breakpoint == 0 :
        # 현재까지 리스트 개수 카운트
        if count_temp >= unit_of_scroll:
            count_temp = 0
            ele_posts = browser.find_element_by_class_name('products_container').find_elements_by_class_name('ProductItem')
            cur_posts_count = len(ele_posts)
            print(cur_posts_count)                             # FOR DEBUGGING

        
        # 목표 갯수에 도달하지 못했을 경우 더 스크롤 (단위로)
        if cur_posts_count < num :
            last_height = browser.execute_script("return document.body.scrollHeight")
            for _ in range(unit_of_scroll):
                body.send_keys(Keys.END);     time.sleep(1)
                count_temp += 1
            new_height = browser.execute_script("return document.body.scrollHeight")
        
        #목표 갯수 미만 & 스크롤이 더 이상 안 될 때
        if last_height == new_height :
            body.send_keys(Keys.PAGE_UP);     time.sleep(2)
            body.send_keys(Keys.END);             time.sleep(5)
            count_nolonger += 1
        else : count_nolonger = 0
        if count_nolonger > 10 : breakpoint = 1
        
        #목표에 도달 시
        if cur_posts_count >= num : breakpoint = 1
    
    # 스크롤 완료 후
    for ele in ele_posts :
        image_container = ele.find_element_by_class_name('image_container')
        info_container = ele.find_element_by_class_name('info_container')
        product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
        if product_url not in product_set:
            dict_post = {"product_url": product_url}
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
            ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
            dict_post["img_url"] = ele_img.get_attribute("src")
            dict_post['category'] = category
            dict_post["channel"] = info_container.find_element_by_class_name('text_container').find_element_by_tag_name('a').text
            dict_post['product_name'] = info_container.find_element_by_class_name('product_name').find_element_by_tag_name('a').text
            price_list_ele = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')
            dict_post['price_consumer'] = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')[-1].text
            if len(price_list_ele) >= 2 :
                dict_post['price_discount'] = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')[0].text
            
            product_set.add(product_url)
            posts.append(dict_post)
    
    
#    total_posts.extend(posts)
#    posts = []  # initialize posts in one url element
    
    return posts


def output(data, filepath,nowtime=False):
    out = json.dumps(data, ensure_ascii=False)
    
    if nowtime==True:
        now = time.localtime()
        filepath_temp = filepath.split('.json')
        filepath = ''.join(filepath_temp[-1]) + "_%02d%02d%02d_%02d%02d%02d.json" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
#        os.chmod(filepath, 0o777)        # for permission
    else:
        print(out)

        
def output_to_csv(data, filepath, fieldnames, nowtime=False) :
    if nowtime==True:
        now = time.localtime()
        filepath_temp = filepath.split('.csv')
        filepath = ''.join(filepath_temp[-1]) + "_%02d%02d%02d_%02d%02d%02d.csv" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            writer = csv.DictWriter(file, fieldnames, delimiter = ';')
            writer.writeheader()
            for row in data:
                writer.writerow(row)
    else:
        print(data)

In [121]:
data_temp_100 = get_products(root_urls, 100, 20)

C:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


In [127]:
len(data_temp_100)

300

In [182]:
data_temp_100[:2]

[{'product_url': 'https://www.seoulstore.com/products/1205054/detail',
  'img_url': 'https://images.seoulstore.com/products/03b5847cfda1300fc364d32ac1562c2b.jpg?d=640xauto',
  'category': '상의',
  'channel': '꼼파뇨',
  'product_name': '[16수] 핸드포크 타투 반팔 화이트',
  'price_discount': '34% 29,700'},
 {'product_url': 'https://www.seoulstore.com/products/1205043/detail',
  'img_url': 'https://images.seoulstore.com/products/4d1195ce714fee69a64ba61f4ccebbd9.jpg?d=640xauto',
  'category': '상의',
  'channel': '꼼파뇨',
  'product_name': '[16수] LA MIA PIZZA 반팔 크림',
  'price_discount': '34% 29,700'}]

In [186]:
data_temp_100_json[:500]

'[{"product_url": "https://www.seoulstore.com/products/1205054/detail", "img_url": "https://images.seoulstore.com/products/03b5847cfda1300fc364d32ac1562c2b.jpg?d=640xauto", "category": "상의", "channel": "꼼파뇨", "product_name": "[16수] 핸드포크 타투 반팔 화이트", "price_discount": "34% 29,700"}, {"product_url": "https://www.seoulstore.com/products/1205043/detail", "img_url": "https://images.seoulstore.com/products/4d1195ce714fee69a64ba61f4ccebbd9.jpg?d=640xauto", "category": "상의", "channel": "꼼파뇨", "product_nam'

In [20]:
len(data)

30

In [ ]:
# 서울스토어가 처음 페이지 로딩 시 상품페이지 자체가 로딩 안 되는 현상 발견
# 어느 정도 스크롤 후, 로딩할 리스트가 더 있음에도 스크롤이 안 되는 현상 발견
# 따라서 스크롤 방식을 위에서 아래로 변경. 
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.END)

In [166]:
output(get_products(root_urls, 100, 60), 'tong_100',True)

C:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


In [7]:
data_temp_1 = get_products(root_urls[0], 3000, 300)
output(data_temp_1, 'tong_',True)
#실패. 도달 시 while 못 빠져나오나

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


1380
1440
1500
1572
1632
1704
1764
1824
1884
1944
2004
2064
2136
2196
2256
2316
2376
2448
2508
2568
2628
2688
2748
2808
2868
2928
3000


KeyboardInterrupt: 

In [20]:
data_temp_1 = get_products(root_urls[0], 3000, 300)
#output_to_csv(get_products(root_urls, 3000, 300), 'tong_3000', ["product_url", "img_url", "category", "channel", "product_name", "price_consumer", "price_discount"], True)
output(data_temp_1, 'tong_',True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


2172
2232
2292
2352
2412
2472
2544
2604
2664
2724
2784
2844
2904
2964
3024


KeyboardInterrupt: 

In [21]:
data_temp_1 = get_products(root_urls[0], 3000, 300)
#output_to_csv(get_products(root_urls, 3000, 300), 'tong_3000', ["product_url", "img_url", "category", "channel", "product_name", "price_consumer", "price_discount"], True)
output(data_temp_1, 'tong_',True)

NameError: name 'data_temp_1' is not defined

In [ ]:
# 스크롤 완료 후 DB 저장 -> 다른 변수 지우기 전에 백업
    # 스크롤 완료 후
    for ele in ele_posts :
        image_container = ele.find_element_by_class_name('image_container')
        info_container = ele.find_element_by_class_name('info_container')
        product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
        if product_url not in product_set:
            dict_post = {"product_url": product_url}
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
            ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
            dict_post["img_url"] = ele_img.get_attribute("src")
            dict_post['category'] = category
            dict_post["channel"] = info_container.find_element_by_class_name('text_container').find_element_by_tag_name('a').text
            dict_post['product_name'] = info_container.find_element_by_class_name('product_name').find_element_by_tag_name('a').text
            price_list_ele = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')
            dict_post['price_consumer'] = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')[-1].text
            if len(price_list_ele) >= 2 :
                dict_post['price_discount'] = info_container.find_element_by_class_name('price').find_elements_by_css_selector('span')[0].text
            
            product_set.add(product_url)
            posts.append(dict_post)
    
    
#    total_posts.extend(posts)
#    posts = []  # initialize posts in one url element

In [24]:
def output(data, filepath,nowtime=False):
    out = json.dumps(data, ensure_ascii=False)
    
    if nowtime==True:
        now = time.localtime()
        filepath_temp = filepath.split('.json')
        filepath = ''.join(filepath_temp[-1]) + "_%02d%02d%02d_%02d%02d%02d.json" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
#        os.chmod(filepath, 0o777)        # for permission
    else:
        print(out)

In [32]:
url = root_urls[0]
num = 3000
wait_time = 300

In [33]:
def output(data, filepath,nowtime=False):
    out = json.dumps(data, ensure_ascii=False)
    
    if nowtime==True:
        now = time.localtime()
        filepath_temp = filepath.split('.json')
        filepath = ''.join(filepath_temp[-1]) + "_%02d%02d%02d_%02d%02d%02d.json" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
#        os.chmod(filepath, 0o777)        # for permission
    else:
        print(out)
        


"""
Arguments:
- urls: 리스트. 크롤링할 url들을 원소로 함.
- num: url당 크롤링할 아이템 수
- wait_time: 로딩을 몇 초까지 기다릴지

Return:
- total_posts: 리스트. 아이템 당 {"product_url": 상품url, "img_url": 이미지url} 정보를 크롤링한 것을 원소로 함.

크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
"""

total_posts = []         # 전체 크롤링 결과를 저장할 리스트
product_set = set()   # 중복 크롤링 거르기 위한 셋. product_url을 원소로 함
unit_of_scroll = 10      # 반복할 스크롤 단위
count_nolonger = 0  # 목표 갯수 미만 & 스크롤이 더 이상 안 될 때를 대비한 카운트 변수


# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨김
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(
    '--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
# 크롬 브라우저 실행
browser = webdriver.Chrome(executable_path='chromedriver_81', chrome_options=chrome_options)
# 대기가능한 최대 시간
wait = WebDriverWait(browser, wait_time)

posts = []  # 당해 url 크롤링 결과를 저장할 리스트

browser.get(url)
body = browser.find_element_by_tag_name('body')

#카테고리 값
category = url.split('categories/')[-1].split('/')[0]
#wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'image_container')))
#category = browser.find_element_by_class_name('Category_Nav').find_elements_by_class_name('active')[0].text
    
    
# 초기에 목표값만큼 스크롤 - 갈수록 속도가 느려져서 슬립타임 세분화
count_temp = 0
for i in range(int(num // 12 * 1.5)):
    body.send_keys(Keys.END)
    if i < 40 : time.sleep(0.5)
    elif 40<=i<200 : time.sleep(1)
    else : time.sleep(2)
    count_temp += 1

# 목표 갯수 도달까지 스크롤
breakpoint = 0
while len(posts) < num and breakpoint == 0 :
    # 현재까지 리스트 개수 카운트
    if count_temp >= unit_of_scroll:
        count_temp = 0
        ele_posts = browser.find_element_by_class_name('products_container').find_elements_by_class_name('ProductItem')
        cur_posts_count = len(ele_posts)
        print(cur_posts_count)                             # FOR DEBUGGING

    
    # 목표 갯수에 도달하지 못했을 경우 더 스크롤 (단위로)
    if cur_posts_count < num :
        print('목표 갯수에 도달하지 못했을 경우 더 스크롤_start')
        last_height = browser.execute_script("return document.body.scrollHeight")
        for _ in range(unit_of_scroll):
            body.send_keys(Keys.END);     time.sleep(1)
            count_temp += 1
        new_height = browser.execute_script("return document.body.scrollHeight")
    
        #목표 갯수 미만 & 스크롤이 더 이상 안 될 때
        if last_height == new_height :
            body.send_keys(Keys.PAGE_UP);     time.sleep(2)
            body.send_keys(Keys.END);             time.sleep(5)
            count_nolonger += 1
        else : count_nolonger = 0
        if count_nolonger > 10 : breakpoint = 1
        print('목표 갯수에 도달하지 못했을 경우 더 스크롤_end')
    
    #목표에 도달 시
    else : breakpoint = 1; print('목표에 도달 함')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: use options instead of chrome_options


2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2580
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2640
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2700
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2760
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2820
목표 갯수에 도달하지 못헀을 경우 더 스크롤_start
목표 갯수에 도달하지 못헀을 경우 더 스크롤_end
2892
목표 갯수에 도달하지 못헀을 경우 더

In [34]:
# 스크롤 완료 후
total_crawling_count = len(ele_posts)

for idx, ele in enumerate(ele_posts) :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    if product_url not in product_set:
        dict_post = {"product_url": product_url}
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
        ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
        dict_post["img_url"] = ele_img.get_attribute("src")
        dict_post['category'] = category
        
        product_set.add(product_url)
        if idx < (total_crawling_count//2) :
            posts.append(dict_post)
        else :
            posts_2.append(dict_post)

output(posts, 'tong_3000_1stfile_',True)
output(posts_2, 'tong_3000_2ndfile_',True)

KeyboardInterrupt: 

In [38]:
len(ele_posts)

3012

In [41]:
for idx, ele in enumerate(ele_posts) :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    if product_url not in product_set:
        dict_post = {"product_url": product_url}
#        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
        ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
        dict_post["img_url"] = ele_img.get_attribute("src")
        dict_post['category'] = category
        
        product_set.add(product_url)
        if idx < (total_crawling_count//2) :
            posts.append(dict_post)
        else :
            posts_2.append(dict_post)

KeyboardInterrupt: 

In [42]:
print(ele_posts[0].find_element_by_class_name('image_container').find_element_by_tag_name('a').get_attribute('href'))
print(ele_posts[0].find_element_by_class_name('image_container').find_element_by_class_name('ImageLoader.ratio_1_1.loaded').get_attribute("src"))
#print(ele_posts[0].find_element_by_class_name('info_container'))

https://www.seoulstore.com/products/1205502/detail
https://images.seoulstore.com/products/0398f69e67849c0f62fb2dc1e75c34cb.jpg?d=640xauto
<selenium.webdriver.remote.webelement.WebElement (session="a9ee4149988a26ff202adff541aeb0d9", element="f0e8d137-a3e7-4f5f-92d8-44fa531a439e")>


In [46]:
total_crawling_count = len(ele_posts)
posts = []  #initialize
posts_2 = []  #initialize
ele_posts_1st = ele_posts[:total_crawling_count//2]
ele_posts_2nd = ele_posts[total_crawling_count//2:]


for ele in ele_posts_1st :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    
    dict_post = {"product_url": product_url}
#    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
    ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
    dict_post["img_url"] = ele_img.get_attribute("src")
    dict_post['category'] = category
    
#    product_set.add(product_url)
    posts.append(dict_post)
output(posts, 'tong_3000_1stfile_',True)
#output(posts_2, 'tong_3000_2ndfile_',True)

KeyboardInterrupt: 

In [ ]:
for idx, ele in enumerate(ele_posts_2nd) :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    
    dict_post = {"product_url": product_url}
#    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
    ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
    dict_post["img_url"] = ele_img.get_attribute("src")
    dict_post['category'] = category
    
#    product_set.add(product_url)
    posts.append(dict_post)
output(posts_2, 'tong_3000_2ndfile_',True)

In [54]:
posts = []
for ele in ele_posts :
    image_container = ele.find_element_by_class_name('image_container')
    info_container = ele.find_element_by_class_name('info_container')
    product_url = image_container.find_element_by_tag_name('a').get_attribute('href')
    
    dict_post = {"product_url": product_url}
#    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
    ele_img = image_container.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
    dict_post["img_url"] = ele_img.get_attribute("src")
    dict_post['category'] = category
    posts.append(dict_post)
output(posts, 'tong_3000_top_',True)

In [29]:
len(posts)

1415